# APRIORI

## ALGORITMO DE ASSOCIAÇÃO

# LIB's

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import datetime

from apyori import apriori

# DATASET

In [2]:
df = pd.read_csv('data.csv', sep = ';', decimal = ',')
df.head()

,data_pedido,hora_pedido,tipo_entrega,tempo_minutos,borda,refrigerante,Sobremesa,valor_pizza,valor_total
0,29/10/2011,21:00:00,Entrega,55,N,N,S,38.5,38.5
1,29/10/2011,21:10:00,Entrega,45,N,N,S,38.5,38.5
2,30/10/2011,20:17:00,Entrega,51,N,N,S,38.5,38.5
3,01/11/2011,21:10:00,Entrega,50,N,S,S,34.0,38.5
4,04/11/2011,20:37:00,Entrega,33,N,S,S,34.0,38.5


In [3]:
df.shape

(1000, 9)

In [4]:
df.dtypes

data_pedido       object
hora_pedido       object
tipo_entrega      object
tempo_minutos      int64
borda             object
refrigerante      object
Sobremesa         object
valor_pizza      float64
valor_total      float64
dtype: object

## VARIÁVEIS DE TEMPO

In [5]:
# alterando o tipo de dado da variavel data_pedido
df['data_pedido'] = pd.to_datetime(df['data_pedido'], format = ('%d/%m/%Y'))

# criando coluna dia da semana
df['dia_semana'] = df['data_pedido'].dt.day_name()

In [6]:
# horário dos pedidos
print(df['hora_pedido'].min(), df['hora_pedido'].max())

18:00:00 23:42:00


In [7]:
# 'extraindo' somente a hora
df['horario'] = df['hora_pedido'].str.slice(start=0, stop = 2)

#contando a qt de ocorrencias
df['horario'].value_counts()

20    315
21    241
19    166
22    130
18    101
23     47
Name: horario, dtype: int64

In [8]:
# notamos que a pizzaria funciona por 6 horas. Dividindo por cada duas horas,
# podemos categorizar os horários como horário de pico alto, medio e baixo

pico_dict = ({ '20' : 'pico - alto',
             '21' : 'pico - alto',
             '19' : 'pico - medio',
             '22' : 'pico - medio',
             '18' : 'pico - baixo',
             '23' : 'pico - baixo'})

df['horario'] = df['horario'].map(pico_dict)

df['horario'].value_counts()

pico - alto     556
pico - medio    296
pico - baixo    148
Name: horario, dtype: int64

## VARIÁVEL 'BORDA'

In [10]:
df['borda'].value_counts()

N          849
S          126
Borda S     18
Borda N      7
Name: borda, dtype: int64

In [14]:
# tratando a variável borda
df['borda'] = df['borda'].apply(lambda x: 'N' if x == 'Borda N'
                 else x)

df['borda'] = df['borda'].apply(lambda x: 'S' if x == 'Borda S'
                 else x)